In [13]:
print(111)

111


In [1]:
import pandas as pd

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('pandasToSparkDF').getOrCreate()

from pyspark.sql.functions import mean, col, split, regexp_extract, when, lit

from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, QuantileDiscretizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [3]:
import pandas as pd

In [4]:
from pyspark.sql.functions import pandas_udf

In [14]:
from pyspark.sql.types import *
@pandas_udf(IntegerType())
def slen(s: pd.Series) -> pd.Series:
    return s.str.len()

In [6]:
from pyspark.sql.types import *
@pandas_udf(IntegerType())
def slen(s: pd.Series) -> pd.Series:
    return s.str.len()

In [7]:
@pandas_udf("string")
def to_upper(s: pd.Series) -> pd.Series:
    return s.str.upper()

In [8]:
df = spark.createDataFrame([("John Doe",)], ("name",))
df.select(to_upper("name")).show()

+--------------+
|to_upper(name)|
+--------------+
|      JOHN DOE|
+--------------+



In [10]:
@pandas_udf("first string, last string")
def split_expand(s: pd.Series) -> pd.DataFrame:
    return s.str.split(expand=True)
df = spark.createDataFrame([("John Doe",)], ("name",))
df.select(split_expand("name").alias("show")).show()

+-----------+
|       show|
+-----------+
|{John, Doe}|
+-----------+



In [11]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StructField, FloatType

schema = StructType([
    StructField("foo", FloatType(), False),
    StructField("bar", FloatType(), False)
])

In [18]:
#返回多重树值
def udf_test(n):
    return (n / 2, n % 2) if n and n != 0.0 else (float('nan'), float('nan'))

test_udf = udf(udf_test, schema)
df = spark.createDataFrame(([(1, 2.0), (2, 3.0)]),("x","y"))

foobars = df.select(test_udf("y").alias("foobar"))
foobars.printSchema()
foobars.show()

root
 |-- foobar: struct (nullable = true)
 |    |-- foo: float (nullable = false)
 |    |-- bar: float (nullable = false)

+----------+
|    foobar|
+----------+
|{1.0, 0.0}|
|{1.5, 1.0}|
+----------+



In [37]:
#多重返回dataframe
d_struct = StructType([
    StructField("foo", FloatType(), False),
    StructField("bar", FloatType(), False)
])


@udf(schema)
def udf_test_dataframe(n)-> pd.DataFrame:
    print(n)
    print(type(n))
    print(111111)
    return n+22,n+1

#dataframe_udf = udf(udf_test_dataframe, schema)
df = spark.createDataFrame(([(1, 2.0), (2, 3.0)]),("x","y"))

foobars = df.select(udf_test_dataframe("y").alias("aaa"))
foobars.printSchema()
foobars.show()

root
 |-- aaa: struct (nullable = true)
 |    |-- foo: float (nullable = false)
 |    |-- bar: float (nullable = false)

+-----------+
|        aaa|
+-----------+
|{24.0, 3.0}|
|{25.0, 4.0}|
+-----------+



In [47]:
#多重返回dataframe
d_struct = StructType([
    StructField("foo", FloatType(), False),
    StructField("bar", FloatType(), False)
])


@udf(schema)
def udf_test_dataframe(n)-> pd.DataFrame:
    print(n)
    print(type(n))
    print(111111)
    return n+22,n+1

#dataframe_udf = udf(udf_test_dataframe, schema)
df = spark.createDataFrame(([(1, 2.0), (2, 3.0)]),("x","y"))

#create virtual table. talbe columns is footbar.foo  And footbar.bar
foobars = df.select(udf_test_dataframe("y").alias("footbar"))
#foobars=df.withColumn('udf_results',udf_test_dataframe)  
foobars.printSchema()
foobars.select("footbar.foo","footbar.bar").show()

root
 |-- footbar: struct (nullable = true)
 |    |-- foo: float (nullable = false)
 |    |-- bar: float (nullable = false)

+----+---+
| foo|bar|
+----+---+
|24.0|3.0|
|25.0|4.0|
+----+---+



In [35]:
##多个参数
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf

def sum(x, y):
    return x + y

sum_cols = udf(sum, IntegerType())

a=spark.createDataFrame([(101, 1, 16)], ['ID', 'A', 'B'])
a.show()
a.withColumn('Result', sum_cols('A', 'B')).show()


+---+---+---+
| ID|  A|  B|
+---+---+---+
|101|  1| 16|
+---+---+---+

+---+---+---+------+
| ID|  A|  B|Result|
+---+---+---+------+
|101|  1| 16|    17|
+---+---+---+------+



In [52]:
#多个参数 多重返回dataframe
d_struct = StructType([
    StructField("foo", FloatType(), False),
    StructField("bar", FloatType(), False)
])


@udf(schema)
def udf_test_dataframe(n,a)-> pd.DataFrame:
    print(n)
    print(type(n))
    print(111111)
    return n+22,n+1

#dataframe_udf = udf(udf_test_dataframe, schema)
df = spark.createDataFrame(([(1, 2.0), (2, 3.0)]),("x","y"))

#create virtual table. talbe columns is footbar.foo  And footbar.bar
foobars = df.select(udf_test_dataframe("y","x").alias("footbar"))
#foobars=df.withColumn('udf_results',udf_test_dataframe)  
foobars.printSchema()
foobars.select("footbar.foo","footbar.bar").show()

root
 |-- footbar: struct (nullable = true)
 |    |-- foo: float (nullable = false)
 |    |-- bar: float (nullable = false)

+----+---+
| foo|bar|
+----+---+
|24.0|3.0|
|25.0|4.0|
+----+---+

